<a href="https://colab.research.google.com/github/chrispinheiro/techchallenge_f3/blob/main/TechChallenge_f3_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Conexão com o Google Drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Não executar, arquivo já disponível
# obter title e content
import json

# Caminho para o arquivo JSON original
input_file = '/content/drive/MyDrive/techchallenge/LF-Amazon-part-23 1.json'

# Caminho para o novo arquivo JSON
output_file = '/content/drive/MyDrive/techchallenge/products.json'

# Abrindo o arquivo JSON original
with open(input_file, 'r', encoding='utf-8') as f:
    try:
        # Carregando o arquivo JSON inteiro (caso seja uma lista de objetos)
        data = json.load(f)
    except json.JSONDecodeError as e:
        print(f"Erro ao carregar o arquivo JSON: {e}")
        data = []

# Filtrando os dados e ignorando aqueles que causam erro durante o processo
filtered_data = []
for item in data:
    try:
        # Obter dados das colunas 'title' e 'content'
        if item.get('title') and item.get('content'):
            filtered_data.append({
                'title': item['title'].replace('"',''),
                'content': item['content'].replace('"','')
            })
    except KeyError as e:
        # Ignorar registros que tenham problema de chave
        print(f"Registro ignorado devido a erro de chave: {e}")
    except Exception as e:
        # Ignorar outros erros genéricos
        print(f"Registro ignorado devido a erro: {e}")

# Salvando os dados filtrados em um novo arquivo JSON
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(filtered_data, f, ensure_ascii=False, indent=4)

print(f"Novo arquivo JSON gerado em: {output_file}")


In [2]:
!pip install openai
!pip install requests

In [3]:
#Importar bibliotecas
import pandas as pd
import json
import requests

In [4]:
from openai import OpenAI
# Substitua 'your_openai_api_key' pela sua chave de API da OpenAI
client = OpenAI(api_key='')


In [5]:
# Gerar resumo para cada descrição usando Chat GPT
# Limitando número de consultas por conta do tempo
def summarize_product(product_file):
    summaries = []
    item = 0

    # Carrega o conteúdo das notícias de um arquivo JSON
    with open(product_file, 'r') as file:
      product_data = json.load(file)

    for produto in product_data:
      item = item + 1

      product = produto['title']
      content = produto['content']

      # Define os dados para enviar para a API da GPT-3.5-turbo
      response = client.chat.completions.create(
          model="gpt-3.5-turbo",
          response_format={ "type": "json_object" },
          messages=[
            {
              "role": "system",
              "content": "Summarize the content of the product and return in JSON format containing only the summary as the value of the 'content' key."
            },
            {
              "role": "user",
              "content": f"{content}\n###"
            }
          ],
          temperature=0.3,
          max_tokens=64,
          top_p=1
        )

      # Extrai o resumo da resposta da API
      summary_text = response.choices[0].message.content.strip()

      summaries.append({
          "title": produto['title'],
          "content": produto['content'],
          "summary": summary_text.replace('{\n    \"content\": \"', "")
      })
      # if item % 100 == 0:
      #   print('Item: ',item)

      # if item == 2000:
      #   break


    # Salva os resultados em um arquivo JSON
    with open('/content/drive/MyDrive/techchallenge/product_summaries.json', 'w') as json_file:
      json.dump({"product_summaries": summaries}, json_file)

    print("final")

# Chame a função com o caminho para seu arquivo de conteúdo de notícias
summarize_product('/content/drive/MyDrive/techchallenge/products.json')

final


In [6]:
#formatando arquivo json
import json
amaz = '/content/drive/MyDrive/techchallenge/product_summaries.json'

# Criando o arquivo dados.json
with open('/content/drive/MyDrive/techchallenge/dados.json', 'w') as arquivo:
  with open(amaz, 'r', encoding='utf-8') as infile:
    for linha in infile:
        json.dump(linha, arquivo)
        arquivo.write('\n')
# Cada objeto JSON em uma nova linha


In [7]:
def process_product_summaries_file(file_path, processed_data):
    #Lê um arquivo JSON do nosso dataset, processa cada produto para formatar conforme o solicitado e adiciona à lista processed_data
    with open(file_path, 'r', encoding='utf-8') as file:
        json_data = json.load(file)
        #news_list = json_data["news_summaries"] ATENCAO
        product_list = json_data["product_summaries"]


        for item in product_list:
            title = item["title"]
            content = item["content"]
            summary = item["summary"]
            formatted_text = f"SUMMARIZE THIS PRODUCTS.\n[|Products|] {title}[|eProducts|]\n\n[|content|]{content}[|econtent|]\n\n[|summary|]{summary}[|esummary|]"
            processed_data.append({"input": formatted_text})



In [8]:
# Lista para armazenar todos os dados processados
processed_data = []

# Adicionar dados processados do arquivo JSON regular à mesma lista
process_product_summaries_file(r'/content/drive/MyDrive/techchallenge/product_summaries.json', processed_data)

# Salvar todos os dados processados em um arquivo JSON
output_filename = r'/content/drive/MyDrive/techchallenge/product_dataset_chat_data.json'
with open(output_filename, 'w', encoding='utf-8') as file:
    json.dump(processed_data, file, ensure_ascii=False, indent=4)

print(f"Todos os dados reformatados foram salvos em '{output_filename}'.")


Todos os dados reformatados foram salvos em '/content/drive/MyDrive/techchallenge/product_dataset_chat_data.json'.


In [9]:
!pip install triton

!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets


  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-gpsc9ajj/unsloth_29a66536ea8942d1b89db6ec728eb5ee
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-gpsc9ajj/unsloth_29a66536ea8942d1b89db6ec728eb5ee
  Resolved https://github.com/unslothai/unsloth.git to commit ae9e264e33c69b53dd5d533a4c5a264af4141c28
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [10]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

DATA_PATH = "/content/drive/MyDrive/techchallenge/product_dataset_chat_data.json"
OUTPUT_PATH_DATASET = "/content/drive/MyDrive/techchallenge/formatted_product_dataset_chat_data.json"
max_seq_length = 2048
dtype = None
load_in_4bit = True
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [11]:
def format_dataset_into_model_input(data):
    def separate_text(full_text):
        product_start = full_text.find("[|Products|]") + len("[|Products|]")
        product_end = full_text.find("[|eProducts|]")
        content_start = full_text.find("[|content|]") + len("[|content|]")
        content_end = full_text.find("[|econtent|]")
        summary_start = full_text.find("[|summary|]") + len("[|summary|]")
        summary_end = full_text.find("[|esummary|]")

        instruction = full_text.split('\n')[0]
        product = full_text[product_start:product_end].strip()
        input_text = full_text[content_start:content_end].strip()
        response = full_text[summary_start:summary_end].strip()

        return instruction, product, input_text, response

    # Inicializando as listas para armazenar os dados
    instructions = []
    products = []
    inputs = []
    outputs = []

    # Processando o dataset
    #for prompt in data['train']['input']:
    for prompt in data['input']:
      instruction, product, input_text, response = separate_text(prompt)

      instructions.append('SUMMARIZE THE CONTENT OF THIS PRODUCT')
      products.append(product)
      inputs.append(input_text)
      outputs.append(response)

    # Criando o dicionário final
    formatted_data = {
        "instruction": instructions,
        "product": products,
        "input": inputs,
        "output": outputs
    }

    # Salvando o resultado em um arquivo JSON
    with open(OUTPUT_PATH_DATASET, 'w') as output_file:
        json.dump(formatted_data, output_file, indent=4)

    print(f"Dataset salvo em {OUTPUT_PATH_DATASET}")

In [12]:
dados_train = load_dataset('json', data_files=DATA_PATH, split='train')
format_dataset_into_model_input(dados_train)

Generating train split: 0 examples [00:00, ? examples/s]

Dataset salvo em /content/drive/MyDrive/techchallenge/formatted_product_dataset_chat_data.json


In [13]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)


==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [14]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.9.post4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [15]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):

        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset

OUTPUT_PATH_DATASET = "/content/drive/MyDrive/techchallenge/formatted_product_dataset_chat_data.json"

dataset = load_dataset("json", data_files=OUTPUT_PATH_DATASET, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [16]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [17]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1 | Num Epochs = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,0.467800
2,0.467800
3,0.463500
4,0.447600
5,0.424400
6,0.393300
7,0.352600
8,0.309400
9,0.264400
10,0.226400


In [24]:
#Teste 01
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "SUMMARIZE THIS PRODUCT",
        "VW, Audi Diesel 1.9 2.0 Dohc Tdi Pd Engine Timing Tools",
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nSUMMARIZE THIS PRODUCT\n\n### Input:\nVW, Audi Diesel 1.9 2.0 Dohc Tdi Pd Engine Timing Tools\n\n### Response:\nThis product is a set of tools that is specially designed for the VW and Audi 1.9 and 2.0 litre TDI PD DOHC engines. It provides a comprehensive toolset for timing belt replacement and other timing-related services. The tool kit includes a serpentine belt tool, cranklock pin,']

In [25]:
#Teste 02
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "SUMMARIZE THIS PRODUCT",
        "VW, Audi Diesel 1.9 2.0 Dohc Tdi Pd Engine Timing Tools",
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
SUMMARIZE THIS PRODUCT

### Input:
VW, Audi Diesel 1.9 2.0 Dohc Tdi Pd Engine Timing Tools

### Response:
This product is about VW, Audi Diesel 1.9 2.0 Dohc Tdi Pd Engine Timing Tools. It gives a lot of information about this topic. I recommend this article to everyone who is interested in VW, Audi Diesel 1.9 2.0 Dohc Tdi Pd Engine Timing Tools.

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
SUMMARIZE THIS PRODUCT

### Input:
The EXO Bib provides great support and compression, a lightweight stretch upper and printed 'Exo


In [20]:
# Salvando modelo e tokenizador
model.save_pretrained("/content/drive/MyDrive/techchallenge/lora_model") # Local saving
tokenizer.save_pretrained("/content/drive/MyDrive/techchallenge/lora_model")

('/content/drive/MyDrive/techchallenge/lora_model/tokenizer_config.json',
 '/content/drive/MyDrive/techchallenge/lora_model/special_tokens_map.json',
 '/content/drive/MyDrive/techchallenge/lora_model/tokenizer.json')

Testes

In [ ]:
!pip install triton

!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets

In [ ]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

max_seq_length = 2048
dtype = None
load_in_4bit = True
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]

In [21]:
import json

product_teste = []

with open('/content/drive/MyDrive/techchallenge/teste_products/products.json','r') as product_file:
  product_teste = json.loads(product_file.read())


In [ ]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "/content/drive/MyDrive/techchallenge/lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)

In [27]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

inputs = tokenizer(
[
alpaca_prompt.format(
        "SUMMARIZE THIS PRODUCT",
        product_teste[4]['content'],
        "",
    )
], return_tensors = "pt").to("cuda")

# outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
# tokenizer.batch_decode(outputs)

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
SUMMARIZE THIS PRODUCT

### Input:
With a pointed bed sheet hat, robe decorated with stars, neck medallion and a glowing ornament on his bo staff, Donnie is The Magnificent Wizard, ready to cast spells in the Turtles' imaginary game. But Donnie is shocked to find out all the Turtles have been transported by an evilwizard to complete his maze. After discovering the princess that needs rescuing is April, Donnie is motivated more than ever to defeat the maze and rescue her.

### Response:
The product that is being described is a new type of laundry detergent that is designed to make it easier to do laundry. It claims to be able to clean fabrics thoroughly, remove stains easily, and freshen the fabric of the laundry. It is suitable for all fabric types, including cotton, linen, and synthetic fabrics.